# 6D Pose Estimation

## Set up the project

We will work with a portion of this dataset, which you can find here: https://drive.google.com/drive/folders/19ivHpaKm9dOrr12fzC8IDFczWRPFxho7

Set some variables to conditionally run some codes. First download the project and change directory to ```6DPose_Estimation```

In [ ]:
MOUNT_DRIVE = False
COMET_ML = False

In [ ]:
if MOUNT_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    %cd /content/drive/MyDrive/6DPose_Estimation/

Install all dependencies of PyTorch dependencies

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
%%capture
import os
import torch

%env TORCH=$torch.__version__
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-spline-conv -f https://data.pyg.org/whl/torch-${TORCH}.html

Install all packages, you may need to restart the runtime before continuing

In [ ]:
!pip install -r ./requirements.txt
print("Restart runtime")

In [ ]:
import os
import yaml
import torch
import torchvision
import open3d as o3d
import itertools
import shutil
from torch.utils.data import Dataset
from torch import nn, optim
import torch.nn.functional as F
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import matplotlib.patches as patches
import wandb
from scipy.spatial.transform import Rotation as R
from torchvision import models
import cv2
from torch.optim import Adam
import quaternion
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
from ultralytics import YOLO
from torchvision.transforms import v2
import trimesh

# install PyTorch Geometric after installation and restart
import torch_geometric
from torch import Tensor
from torch_geometric.nn import knn_interpolate, MessagePassing
from torch_geometric.nn.pool import fps, radius

# import comet-ml
import comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import watch

from utils.data_exploration import load_image
from utils.installation_checker import check_torch_geometric

from data.CustomDatasetPose import IMG_WIDTH, IMG_HEIGHT

# check if everything works
check_torch_geometric()

Set device

In [ ]:
from utils.device_setter import set_device

device = set_device()

## Download dataset

In [ ]:
# Step 1: Download the dataset (LineMOD)
# Download LineMOD dataset
# create directory structure without errors
!mkdir -p datasets/linemod/
%cd datasets/linemod/

In [ ]:
!mkdir -p DenseFusion/
%cd DenseFusion/

In [ ]:
# Download dataset (which includes a portion of the LimeMOD dataset)
!gdown --folder "https://drive.google.com/drive/folders/19ivHpaKm9dOrr12fzC8IDFczWRPFxho7"

In [ ]:
if MOUNT_DRIVE:
    !cp /content/drive/MyDrive/6DPose_Estimation /content/ # move to content for faster access to files
    %cd /content/6DPose_Estimation/datasets/linemod/DenseFusion

!unzip Linemod_preprocessed.zip
!rm Linemod_preprocessed.zip
%cd ../../../ # change directory to 6D_pose_estimation

Get working directory

In [ ]:
path = !pwd
path = path[0]

## Modify Dataset

Copy ground truth files to ```Linemod_preprocessed```

In [ ]:
from utils.data_exploration import get_class_names
from utils.preprocessing import copy_gt_file, change_02gt, quaternion_gt

folder_names = get_class_names()
copy_gt_file(folder_names)

Change ```02_gt.yml``` to take only one object

In [ ]:
change_02gt("./datasets/linemod/DenseFusion/Linemod_preprocessed/02_gt.yml")

Add quaternion

In [ ]:
quaternion_gt("./datasets/linemod/DenseFusion/Linemod_preprocessed")

## Data Exploration

Load an image

In [ ]:
load_image(label=1, object=0)

Check if camera intrinsics is same for all images

In [ ]:
root_path = "./datasets/linemod/DenseFusion/Linemod_preprocessed/data"

from utils.data_exploration import check_cam_K_equal

cam_K = check_cam_K_equal(root_path)

## Define CustomDataset

In [ ]:
from data.CustomDatasetPose import CustomDatasetPose

dataset_root = "./datasets/linemod/DenseFusion/Linemod_preprocessed/"

train_dataset = CustomDatasetPose(dataset_root, split="train", device=device, cam_K = cam_K)
print(f"Training samples: {len(train_dataset)}")

val_dataset = CustomDatasetPose(dataset_root, split="validation", device=device, cam_K = cam_K)
print(f"Validation samples: {len(val_dataset)}")

test_dataset = CustomDatasetPose(dataset_root, split="test", device=device, cam_K = cam_K)
print(f"Testing samples: {len(test_dataset)}")

## Data Preprocessing

Structure the data for YOLO such that
```
datasets/
├── data.yaml
│
├── train/
│   ├── images/
│   │
│   └── labels/
│  
├── val/
│
└── test/
```

In [ ]:
# divide the dataset into training, validation and testing set
train_samples = train_dataset.get_samples_id()
validation_samples = val_dataset.get_samples_id()
test_samples = test_dataset.get_samples_id() # test folder is optional for training YOLO

Create a new folder containing all the info, we just need the rgb image and a text file with the label and bounding box.
The ```Linemod_preprocessed``` is not removed, as it contains info about translation and rotation that are needed for pose estimation, but not for object detection model.

The working directory is in the ```6DPose_Estimation```

Create YOLO yaml

In [ ]:
from utils.preprocessing import create_YOLO_yaml, create_dataset_YOLO

number_classes, class_names = create_YOLO_yaml(path, folder_names)

While creating the folder structure, we have to change the class id by using the index in the array written in the ```data.yaml```

In [ ]:
# create a dictionary to have easily access to the index
index_dict = dict()
for index, el in enumerate(class_names):
    index_dict[int(el)] = index

Create the folders. Note that each image may contain multiple objects. For instance in ```data/02/gt.yml``` for one image there are multiple objects, but just consider the object of that class

In [ ]:
counter_df = create_dataset_YOLO(number_classes, train_samples, validation_samples, test_samples, index_dict, path, train_dataset)

Visualize dataset distribution

In [ ]:
from utils.data_exploration import load_dataset_distribution

load_dataset_distribution(counter_df, index_dict, number_classes)

### Visualize data

Visualize depth image

In [ ]:
from utils.data_exploration import load_depth_image

folder = "02"
object_name = "0101"
img = load_depth_image(f"{path}/datasets/linemod/DenseFusion/Linemod_preprocessed/data/{folder}/depth/{object_name}.png")

Plot the patch of first object of the image, it reads from the ground truth file containing also multiple objects in one image

In [ ]:
from utils.data_exploration import load_depth_patch

load_depth_patch(path, folder, object_name, img)

Get data loader

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)
print(f"Training loader: {len(train_loader)}")
print(f"Validation loader: {len(val_loader)}")
print(f"Test loader: {len(test_loader)}")

Plot one batch of data

In [ ]:
from utils.data_exploration import plot_batch_data

plot_batch_data(train_loader, val_loader, test_loader)

## Training Object Detection model

In [ ]:
from ultralytics import YOLO

model_path = f"{path}/checkpoints/yolo11n.pt"
model = YOLO(model_path)
epochs = 50
batch_size = 64
IMG_SIZE = 640

In [ ]:
# model will automatically scale the image and related bounding box according to imgsz
results = model.train(data=f"{path}/datasets/data.yaml", epochs=epochs, batch=batch_size, device=device,
        imgsz=IMG_SIZE,
        augment=True,
        flipud=0.5,
        fliplr=0.5,
        hsv_h=0.4,
        hsv_s=0.4,
        hsv_v=0.4,
        degrees=120,
        translate=0.1,
        scale=0.5,
        shear=20,
        perspective=0.0001
    )

Copy model file to ```checkpoints```

In [ ]:
shutil.copy(f"./runs/detect/train/weights/best.pt", f"../../../checkpoints/best.pt")

Validate model

In [ ]:
model_path = "../../../checkpoints/best.pt"
model = YOLO(model_path)
results = model.val(
        data=f"{path}/datasets/data.yaml",
        epochs=epochs,
        batch=batch_size,
        imgsz=IMG_SIZE,
        device=device
    )

Test model

In [ ]:
model_path = "../../../checkpoints/best.pt"
model = YOLO(model_path)
results = model.val(
        data=f"{path}/datasets/data.yaml",
        epochs=epochs,
        batch=batch_size,
        imgsz=IMG_SIZE,
        device=device,
        split="test"
    )

## Pose Estimator Module

In [ ]:
%cd ../

In [ ]:
path = !pwd
path = path[0]

In [ ]:
from tqdm import tqdm

os.makedirs("./YOLO/datasets_cropped", exist_ok=True)

# input: path YOLO/datasets
# output: save images

def mask_outside_bbox(path="./YOLO/datasets", color=(0, 0, 0)):

    for folder in os.scandir(path):
        if folder.is_dir():
            os.makedirs(f"./YOLO/datasets_cropped/{folder.name}", exist_ok=True)
            path_dir= os.path.join(path, folder.name)
            images_path = os.path.join(path_dir, "images")
            labels_path = os.path.join(path_dir, "labels")

            for img_name, label_name in zip(sorted(os.listdir(images_path)), sorted(os.listdir(labels_path))):
                img_path = os.path.join(images_path, img_name)
                label_path = os.path.join(labels_path, label_name)
                img = cv2.imread(img_path)
                
                with open(label_path, "r") as f:
                    _, x_center, y_center, w, h = map(float,f.readline().split(" "))
                f.close()
                
                img_height, img_width, _ = img.shape
                
                x1 = round((x_center - w / 2) * img_width)
                y1 = round((y_center - h / 2) * img_height)
                x2 = round((x_center + w / 2) * img_width)
                y2 = round((y_center + h / 2) * img_height)
                
                img_masked = np.full_like(img, color)
                img_masked[y1:y2, x1:x2] = img[y1:y2, x1:x2]
                
                cv2.imwrite(f"./YOLO/datasets_cropped/{folder.name}/{img_name}",img=img_masked)   
             
mask_outside_bbox()

Copy the gt.yaml and info.yaml

In [ ]:
for el in class_names:
    shutil.copy(f"./DenseFusion/Linemod_preprocessed/data/{el}/gt.yml", f"./YOLO/datasets_cropped/{el}_gt.yml")
    shutil.copy(f"./DenseFusion/Linemod_preprocessed/data/{el}/info.yml", f"./YOLO/datasets_cropped/{el}_info.yml")

In [ ]:
class CustomDatasetPose(Dataset): # used to load and preprocess data
    def __init__(self, dataset_root, split='train'):
        """
        Args:
            dataset_root (str): Path to the dataset directory.
            split (str): 'train', 'validation' or 'test'.
        """
        self.dataset_root = dataset_root
        if split == "train":
            self.split = split
        elif split == "validation":
            self.split = "val"
        else:
            self.split = "test"

        # Get list of all samples (folder_id, sample_id)
        self.samples = self.get_all_samples()

        # Check if samples were found
        if not self.samples:
            raise ValueError(f"No samples found in {self.dataset_root}. Check the dataset path and structure.")

        # Define image transformations
        self.transform = transforms.Compose([
            transforms.ToTensor(),
        ])

    def get_samples_id(self):
        return self.samples

    def get_all_samples(self):
        """Retrieve the list of all available sample indices from all folders."""
        for folder in ["train","val","test"]:
            if folder == self.split:
                folder_path = os.path.join(self.dataset_root, f"{folder}")
                #print(folder_path)
                if os.path.exists(folder_path):
                    # get name of files <folder id>_<image>
                    sample_ids = sorted([f.split('.')[0] for f in os.listdir(folder_path) if f.endswith('.png')])
        return sample_ids

    #Define here some usefull functions to access the data
    def load_image(self, img_path):
        """Load an RGB image and convert to tensor."""
        img = Image.open(img_path).convert("RGB")
        return self.transform(img)

    def load_6d_pose(self, sample_id):
        """Load the 6D pose (translation and rotation) for the object in this sample."""
        label = int(sample_id.split("_")[0])
        objectId = int(sample_id.split("_")[1])
        pose_file = os.path.join(self.dataset_root, f"{label:02d}_gt.yml")

        # Load the ground truth poses from the gt.yml file
        with open(pose_file, 'r') as f:
            pose_data = yaml.load(f, Loader=yaml.FullLoader)

        # The pose data is a dictionary where each key corresponds to a frame with pose info
        # We assume sample_id corresponds to the key in pose_data
        if objectId not in pose_data:
            raise KeyError(f"Sample ID {objectId} not found in {label:02d}_gt.yml.")

        for pose in pose_data[objectId]: # There can be more than one pose per sample, but take the one of label=folder_id
            # Extract translation and rotation
            if (int(pose['obj_id']) == int(label)):
                translation = np.array(pose['cam_t_m2c'], dtype=np.float32)  # [3] ---> (x,y,z)
                rotation = np.array(pose['cam_R_m2c'], dtype=np.float32).reshape(3, 3)  # [3x3] ---> rotation matrix
                # bbox is top left corner and width and height info, YOLO needs center coordinates and width and height
                x_min, y_min, width, height = np.array(pose['obj_bb'], dtype=np.float32) # [4] ---> x_min, y_min, width, height
                # compute initial center
                x_center = x_min + width/2
                y_center = y_min + height/2
                
                # store coordinates of the center and width and height of the bounding box normalized to the
                # image width=640 pixels and height=480 pixels
                bbox = np.array([x_center/IMG_WIDTH, y_center/IMG_HEIGHT, width/IMG_WIDTH, height/IMG_HEIGHT], dtype=np.float32)

                obj_id = np.array(pose['obj_id'], dtype=np.float32) # [1] ---> label
                break

        return translation, rotation, bbox, obj_id

    def __len__(self):
        #Return the total number of samples in the selected split.
        return len(self.samples)

    def __getitem__(self, idx):
        #Load a dataset sample.
        sample = self.samples[idx]

        img_path = os.path.join(self.dataset_root, f"{self.split}", f"{sample}.png")

        img = self.load_image(img_path)
        translation, rotation, bbox, obj_id = self.load_6d_pose(sample)

        #Dictionary with all the data
        return {
            "rgb": img,
            "translation": torch.tensor(translation),
            "rotation": torch.tensor(rotation),
            "bbox": torch.tensor(bbox),
            "obj_id": torch.tensor(obj_id),
        }

In [ ]:
dataset_root_pose = "./YOLO/datasets_cropped"

train_dataset = CustomDatasetPose(dataset_root_pose, split="train")
print(f"Training samples: {len(train_dataset)}")

val_dataset = CustomDatasetPose(dataset_root_pose, split="validation")
print(f"Validation samples: {len(val_dataset)}")

test_dataset = CustomDatasetPose(dataset_root_pose, split="test")
print(f"Testing samples: {len(test_dataset)}")

In [ ]:
class PoseEstimatorQuat6D(nn.Module):
    def __init__(self, backbone_name="resnet18", pretrained=True):
        super(PoseEstimatorQuat6D, self).__init__()

        # Load backbone (without last fully connected layer)
        backbone = getattr(models, backbone_name)(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])
        in_features = backbone.fc.in_features

        # 7 output (4 quaternion, 3 translation)
        self.fc = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Linear(256, 7)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = torch.flatten(x, 1)
        output = self.fc(x)

        quat = output[:, :4]
        quat = quat / torch.norm(quat, dim=1, keepdim=True)  # Normalize quaternion

        trans = output[:, 4:]
        return quat, trans

In [ ]:
def criterion(outputs, translation, rotation):
    quat, trans = outputs
    # rotation is list of type quaternion, make it tensor of float32 not float64 (double)
    rotation = torch.tensor(np.stack([el.components for el in rotation]).astype(np.float32)).to(device)
    loss_quat = F.mse_loss(quat, rotation)
    loss_trans = F.mse_loss(trans, translation)

    return loss_quat + loss_trans

In [ ]:
from torch.optim import Adam
import quaternion

def train(model, epoch, dataloader, criterion, optimizer=Adam, device=device):
    model.train()
    running_loss = 0.0

    for batch_idx, data in enumerate(dataloader):
        img, translation, rotation = data["rgb"], data["translation"], data["rotation"]
        img, translation, rotation = img.to(device), translation.to(device), quaternion.from_rotation_matrix(rotation)

        outputs = model(img)
        loss = criterion(outputs, translation, rotation)
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch_idx % 2 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(dataloader.dataset)} '
                f'({100. * batch_idx / len(dataloader):.0f}%)]\t Loss: {loss.item():.6f}')

In [ ]:
learning_rate=[0.001]
batch_size=[2]
num_epoch=10
for lr in learning_rate:
    for batch in batch_size:
        train_dataloader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=batch, shuffle=False)
        model = PoseEstimatorQuat6D().to(device)
        print(model)
        for epoch in range(1, num_epoch):
            train(model, epoch, train_dataloader, criterion, optimizer=Adam(model.parameters(), lr=lr), device=device)

Plot

In [ ]:
import quaternion
import trimesh

In [ ]:
def plotPose(pathImage, translation_gt, rotation_gt, translation_pred, rotation_pred):
    '''
        Input:
            path for image (in DenseFusion)
            ground truth translation tensor (in millimeters)
            ground truth rotation tensor (either matrix or quaternion)
            predicted translation tensor (in millimeters)
            predicted rotation tensor (either matrix or quaternion)
    '''

    # read image
    image = cv2.imread(pathImage)
    transparent_image = image.copy() # copy of the image to work on a transparent image (for the second reference system)

    # read translation and rotation
    rotat_gt = rotation_gt.numpy() # transform tensor to numpy array
    trans_gt = translation_gt.numpy()/1000 # in meters
    rotat_pred = rotation_pred.numpy()
    trans_pred = translation_pred.numpy()/1000

    # read camera intrinsics
    label = pathImage.split("/")[-1].split(".")[0].split("_")[0]
    image_id = pathImage.split("/")[-1].split(".")[0].split("_")[1]
    with open(f"./YOLO/datasets_cropped/{label}_info.yml", 'r') as f:
            camera_info = yaml.load(f, Loader=yaml.FullLoader)
    camera_intrinsics = np.array(camera_info[int(image_id)]["cam_K"]).reshape(3,3)

    # read 3D model
    meshModel = trimesh.load(f"./DenseFusion/Linemod_preprocessed/models/obj_{label}.ply")
    vertices = meshModel.vertices/1000 # it has 3 columns, for X, Y, Z, use unit of measurement of translation
    # compute corners
    min_corner = vertices.min(axis=0) # find for each column the smallest value
    max_corner = vertices.max(axis=0)
    
    bounding_box_3d = np.array([[min_corner[0], min_corner[1], min_corner[2]],
                                [max_corner[0], min_corner[1], min_corner[2]],
                                [max_corner[0], max_corner[1], min_corner[2]],
                                [min_corner[0], max_corner[1], min_corner[2]],
                                [min_corner[0], min_corner[1], max_corner[2]],
                                [max_corner[0], min_corner[1], max_corner[2]],
                                [max_corner[0], max_corner[1], max_corner[2]],
                                [min_corner[0], max_corner[1], max_corner[2]],])

    # convert quaternion to rotation matrix, if input was quaternion
    if rotat_gt.size == 4:
        rotat_gt = quaternion.as_rotation_matrix(np.quaternion(*rotat_gt))
    else:
        rotat_gt = rotat_gt.reshape(3,3)
    if rotat_pred.size == 4:
        rotat_pred = quaternion.as_rotation_matrix(np.quaternion(*rotat_pred))
    else:
        rotat_pred = rotat_pred.reshape(3,3)

    # build 3D axes according to object coordinate system, same unit of measurement of translation, so in meters
    axes_3d = np.array([
        [0, 0, 0],      # origin, in the object coordinate system
        [0.15, 0, 0],   # how long the arrow should be in the X coordinate
        [0, 0.15, 0],   # how long the arrow should be in the Y coordinate
        [0, 0, 0.15]    # how long the arrow should be in the Z coordinate
    ])

    # transform the object coordinate system to the camera coordinate system
    # rotat_gt is 3x3, so axes_3d has to be transposed, then add to origin, and coordinates the translation
    axes_cam_gt = (rotat_gt @ axes_3d.T).T + trans_gt
    # bounding box
    bounding_box_3d_cam_gt = (rotat_gt @ bounding_box_3d.T).T + trans_gt
    # project 3D axes to 2D
    axes_2d_gt = (camera_intrinsics @ axes_cam_gt.T).T # camera_intrinsics is 3x3, while axes_cam_gt 4x3, axes_2d_gt 4x3
    axes_2d_gt = axes_2d_gt[:, :2] / axes_2d_gt[:, 2:3] # take first 2 columns and normalize by depth
    # bounding box
    bounding_box_2d_gt = (camera_intrinsics @ bounding_box_3d_cam_gt.T).T
    bounding_box_2d_gt = (bounding_box_2d_gt[:, :2] / bounding_box_2d_gt[:, 2:3]).astype(int)
    # get point coordinates
    p_gt = [tuple(el) for el in bounding_box_2d_gt]
    # define edges using two points, access with index
    edges = [(0,1), (1,2), (2,3), (3,0), # bottom
             (0,4), (1,5), (2,6), (3,7), # vertical
             (4,5), (5,6), (6,7), (7,4) # top
             ]
    # draw edges
    for el in edges:
        cv2.line(image, p_gt[el[0]], p_gt[el[1]], (0,0,255), 5)

    p0_gt = tuple(axes_2d_gt[0].astype(int)) # take origin coordinates
    p1_gt = tuple(axes_2d_gt[1].astype(int))
    p2_gt = tuple(axes_2d_gt[2].astype(int))
    p3_gt = tuple(axes_2d_gt[3].astype(int))

    # color is in BGR format, set tickness=2
    cv2.arrowedLine(image, p0_gt, p1_gt, (0, 0, 255), 2) # X is red
    cv2.arrowedLine(image, p0_gt, p2_gt, (0, 255, 0), 2) # Y is green
    cv2.arrowedLine(image, p0_gt, p3_gt, (255, 0, 0), 2) # Z is blue

    # for predicted
    axes_cam_pred = (rotat_pred @ axes_3d.T).T + trans_pred
    bounding_box_3d_cam_pred = (rotat_pred @ bounding_box_3d.T).T + trans_pred

    axes_2d_pred = (camera_intrinsics @ axes_cam_pred.T).T
    axes_2d_pred = axes_2d_pred[:, :2] / axes_2d_pred[:, 2:3]
    bounding_box_2d_pred = (camera_intrinsics @ bounding_box_3d_cam_pred.T).T
    bounding_box_2d_pred = (bounding_box_2d_pred[:, :2] / bounding_box_2d_pred[:, 2:3]).astype(int)

    p_pred = [tuple(el) for el in bounding_box_2d_pred]
    edges = [(0,1), (1,2), (2,3), (3,0), # bottom
             (0,4), (1,5), (2,6), (3,7), # vertical
             (4,5), (5,6), (6,7), (7,4) # top
             ]
    for el in edges:
        cv2.line(image, p_pred[el[0]], p_pred[el[1]], (255,0,0), 5)

    p0_pred = tuple(axes_2d_pred[0].astype(int))
    p1_pred = tuple(axes_2d_pred[1].astype(int))
    p2_pred = tuple(axes_2d_pred[2].astype(int))
    p3_pred = tuple(axes_2d_pred[3].astype(int))

    cv2.arrowedLine(transparent_image, p0_pred, p1_pred, (0, 0, 255), 2)
    cv2.arrowedLine(transparent_image, p0_pred, p2_pred, (0, 255, 0), 2)
    cv2.arrowedLine(transparent_image, p0_pred, p3_pred, (255, 0, 0), 2)

    # show image after merging the two images
    overlapImage = cv2.addWeighted(transparent_image, 0.5, image, 1, 0)
    plt.imshow(cv2.cvtColor(overlapImage, cv2.COLOR_BGR2RGB))
    plt.title("Object Pose Estimation (prediction is transparent)")
    plt.show()

    return